Blank


In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

from sklearn.preprocessing import StandardScaler

import pickle


In [30]:
df1 = pd.read_csv('../../Data/complete_fire.csv')

In [31]:
df1.head()

,unique_fire_identifier,lattitude,longitude,acres_burned,fire_cause,incident_name,IsFSAssisted,origin_county,state,year,...,area (acres),population,d0,d1,d2,d3,d4,DSCI_avg,DSCI_summed,precipitation(in)
0,2003-WYCMX-200237,-105.563067,44.603340,139.091837,Unknown,SCHWARTZ,1.0,Campbell,WY,2003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-CABDU-006784,-117.076679,34.733894,1906.698008,Unknown,MENTONE,1.0,San Bernardino,CA,2004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008-IDGBK-000002,-116.216680,43.666673,685.728842,Unknown,NIFC RAMP SUPPORT,1.0,Ada,ID,2008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2008-IDCTF-008902,-111.426912,42.096336,20.000000,Unknown,St. Charles RX,0.0,Bear Lake,ID,2008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2008-AZPNF-000975,-112.072414,34.598306,77.946287,Unknown,Mingus/Cherry Rx,0.0,Yavapai,AZ,2008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df1.dropna(inplace = True)

In [33]:
df1.shape

(167273, 23)

In [34]:
conditions = [df1['acres_burned'] <= .25,
             (df1['acres_burned'] > .25) & (df1['acres_burned'] <= 9.9),
             (df1['acres_burned'] > 9.9) & (df1['acres_burned'] <= 99.99),
             (df1['acres_burned'] > 99.99) & (df1['acres_burned'] <= 299),
             (df1['acres_burned'] > 299) & (df1['acres_burned'] <= 999),
             (df1['acres_burned'] > 999) & (df1['acres_burned'] <= 5000),
             (df1['acres_burned'] > 5000) & (df1['acres_burned'] <= 9999),
             (df1['acres_burned'] > 9999) & (df1['acres_burned'] <= 49_999),
             (df1['acres_burned'] > 49_999) & (df1['acres_burned'] <= 100_000),
              (df1['acres_burned'] > 100_000)
                
             ]
# choices = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
choices = [0,1,2,3,4,5,6,7,8,9]

df1['class'] = np.select(conditions, choices, default = 'A')

In [35]:
df1.drop(columns = ['incident_name', 'unique_fire_identifier', 'IsFSAssisted','DSCI_summed', 'DSCI_avg'], inplace = True)

In [36]:
df1['origin_county'].value_counts(normalize = True)

Los Angeles      0.080396
Riverside        0.020876
Coconino         0.017462
Beltrami         0.012686
Lake             0.011598
                   ...   
Wake             0.000006
Estill           0.000006
Collingsworth    0.000006
Orangeburg       0.000006
Sharkey          0.000006
Name: origin_county, Length: 1335, dtype: float64

In [38]:
rf = RandomForestClassifier()

In [39]:
df1 = pd.get_dummies(df1, columns = [ 'origin_county', 'state'], drop_first = True)

In [40]:
df1.shape

(167273, 1396)

In [41]:
X = df1.drop(columns = ['acres_burned', 'class', 'fire_cause'])
y = df1['class']

In [42]:
columns = list(X.columns)
with open('./columns_list.pkl', 'wb') as pickle_out:
    columns = pickle.dump(columns, pickle_out)

In [16]:
with open('./columns_list2.pkl', 'rb') as pickle_in:
    columns2 = pickle.load(pickle_in)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 5, stratify = y)

In [44]:
rf = RandomForestClassifier(n_estimators = 10)

In [45]:
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10)

In [46]:
preds = rf.predict(X_test)

In [47]:
preds

array(['5', '2', '0', ..., '5', '1', '3'], dtype=object)

In [28]:
X_test.head(25)

,lattitude,longitude,year,month,value,anomaly,area (acres),population,d0,d1,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
196891,-122.283297,40.399655,2021,8,77.1,3.4,104772480.0,39237836.0,0.0000,4.6280,...,0,0,0,0,0,0,0,0,0,0
179328,-100.954180,48.036945,2021,5,52.8,-0.9,45250560.0,774948.0,1.9050,5.1050,...,0,0,0,0,0,0,0,0,0,0
152569,-111.783894,35.320955,2020,8,84.1,6.3,72963840.0,7177986.0,9.0275,35.4700,...,0,0,0,0,0,0,0,0,0,0
148893,-120.464459,46.056950,2020,7,64.8,0.7,45633920.0,7718785.0,13.0600,11.2825,...,0,0,0,0,0,0,1,0,0,0
152135,-116.963699,33.991534,2020,7,76.6,1.9,104772480.0,39499738.0,10.9050,27.0450,...,0,0,0,0,0,0,0,0,0,0
119885,-122.415397,41.424955,2019,8,76.8,3.1,104772480.0,39437610.0,6.2675,0.2725,...,0,0,0,0,0,0,0,0,0,0
69278,-116.196513,43.261796,2017,8,66.9,3.3,53487360.0,1719745.0,30.3900,7.7000,...,0,0,0,0,0,0,0,0,0,0
78248,-110.455111,33.292805,2018,1,47.6,6.9,72963840.0,7164228.0,0.0000,59.7100,...,0,0,0,0,0,0,0,0,0,0
114802,-114.502612,35.752565,2019,7,81.9,2.2,72963840.0,7291843.0,4.7460,0.0000,...,0,0,0,0,0,0,0,0,0,0
193760,-111.032212,40.459146,2021,8,70.9,1.5,54338560.0,3337975.0,0.0000,0.0000,...,0,0,0,1,0,0,0,0,0,0


In [49]:
rf.score(X_train, y_train), rf.score(X_test, y_test)

(0.9695984185438488, 0.6089337382529473)

In [50]:
with open('./rfmodel.pkl', 'wb') as pickle_out:
    pickle_out = pickle.dump(rf, pickle_out)

In [51]:
with open('./rfmodel.pkl', 'rb') as pickle_in:
    pickled = pickle.load(pickle_in)

In [52]:
pickled.predict(X_test)

array(['5', '2', '0', ..., '5', '1', '3'], dtype=object)